Load packages and use tensorflow as backend

In [1]:
#####################################################
# Install a pip package in the current Jupyter kernel
# import system level packages
import sys
#!{sys.executable} -m pip install numpy
#!{sys.executable} -m pip install requests
#import requests

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import keras
from keras.layers import Input, Dense, Lambda, Layer, Activation
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import backend as K
from keras import metrics, optimizers
from keras.callbacks import Callback
from keras import utils

import pydot
import graphviz
from keras.utils import plot_model
from keras_tqdm import TQDMNotebookCallback
#from .tqdm_callback import TQDMNotebookCallback
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from tensorflow.python.client import device_lib

Using TensorFlow backend.


In [2]:
##################################################
#test tensorflow, remember to change the kernel
#using kernel that supports GPU computing
#simple test to confirm tensorflow is actually working

hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

a = tf.constant(10)
b = tf.constant(32)
print("10 + 32 = ", sess.run(a + b))

#manually set the random seed to define a replication
r_seed = 55555

#manually set the number for cross validation
num_cv = 5

print("current random seed is: ", r_seed)

b'Hello, TensorFlow!'
10 + 32 =  42
current random seed is:  55555


In [3]:
#######################################################################################################
#check the system information, check if cuda and gpu computing for tensorflow is installed properly
print("whether tensorflow is built with cuda: ", tf.test.is_built_with_cuda())
print("whether gpu computing is available for tensorflow: ", tf.test.is_gpu_available())
print("using keras version: ", keras.__version__)
print("using tensorflow version: ", tf.__version__)
print("\n")
print("Device details:\n", device_lib.list_local_devices())
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

whether tensorflow is built with cuda:  True
whether gpu computing is available for tensorflow:  True
using keras version:  2.1.3
using tensorflow version:  1.4.1


Device details:
 [name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16304631203960567411
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 989921280
locality {
  bus_id: 1
}
incarnation: 1297242920284457422
physical_device_desc: "device: 0, name: TITAN RTX, pci bus id: 0000:01:00.0, compute capability: 7.5"
]
Default GPU Device: /device:GPU:0


Traditional machine learning methods on compressed data

In [4]:
#########################################################################
#Reading files/documents
compress_path = 'counts_data/counts_data_with_label/TCGA_SARC_(0.2chemo_2labels)VSTnrom_count_expr_clinical_data.txt'
#open(path).readline()
#gene expression RNAseq, Batch effects normalized mRNA data

og_data = pd.read_csv(compress_path, sep = "\t", index_col = 0)
og_data = og_data.dropna(axis='columns')
#ExprAlldata.columns = ["Gene", "Counts"]
print("dimension of the input data: ", og_data.shape)
og_data.head(5)

dimension of the input data:  (65, 12099)


,Ensembl_ID,ENSG00000000003.13,ENSG00000000419.11,ENSG00000000457.12,ENSG00000000460.15,ENSG00000000938.11,ENSG00000000971.14,ENSG00000001036.12,ENSG00000001084.9,ENSG00000001167.13,...,ENSG00000280128.1,ENSG00000280143.1,ENSG00000280206.1,ENSG00000280237.1,ENSG00000280789.1,ENSG00000280798.1,ENSG00000280832.1,ENSG00000281332.1,ENSG00000281649.1,response_group
1,TCGA-3B-A9HI-01A,9.869982,11.189493,9.633288,9.057744,9.087363,12.426148,11.915674,9.696256,10.315499,...,9.323656,7.077178,8.490880,8.804611,8.738330,9.235010,8.087203,7.662738,10.633918,1
3,TCGA-3B-A9HL-01A,11.164872,10.429709,9.187715,8.614128,7.966475,10.053141,12.453439,9.209995,9.841264,...,9.570093,6.574077,8.740703,6.786439,8.330560,8.966090,8.115032,7.395187,10.510393,1
5,TCGA-3B-A9HP-01A,9.614155,10.165624,9.562561,8.163207,8.471740,10.789869,11.155625,8.836156,9.954284,...,9.668858,9.946461,9.038540,11.954611,8.698204,9.541148,8.040706,7.733620,10.819688,1
7,TCGA-3B-A9HR-01A,9.500556,10.769842,9.503313,9.014474,7.309606,7.601647,10.133110,10.971412,10.262254,...,9.677357,8.352724,9.639617,12.058141,8.616255,9.996402,7.454254,8.581126,11.885747,1
8,TCGA-3B-A9HS-01A,10.235303,10.730758,9.064795,9.017946,9.559495,12.268277,12.788385,10.193778,10.303815,...,8.350422,6.645875,8.879500,10.398671,8.856683,9.042467,7.236818,7.125316,11.005164,1


Number of cases in each category

In [5]:
df_count = og_data.groupby('response_group')['Ensembl_ID'].nunique()
print(df_count)
#df_count.nlargest(10)

response_group
0    29
1    36
Name: Ensembl_ID, dtype: int64


In [6]:
###################################################
#store the raw data, and use ensembl id as index
#
df_raw = og_data.iloc[:, 0:]
df_raw = df_raw.set_index('Ensembl_ID')

#notice the last column is the response_group
#df_raw.shape
df_raw.head(3)

,ENSG00000000003.13,ENSG00000000419.11,ENSG00000000457.12,ENSG00000000460.15,ENSG00000000938.11,ENSG00000000971.14,ENSG00000001036.12,ENSG00000001084.9,ENSG00000001167.13,ENSG00000001460.16,...,ENSG00000280128.1,ENSG00000280143.1,ENSG00000280206.1,ENSG00000280237.1,ENSG00000280789.1,ENSG00000280798.1,ENSG00000280832.1,ENSG00000281332.1,ENSG00000281649.1,response_group
Ensembl_ID,,,,,,,,,,,,,,,,,,,,,
TCGA-3B-A9HI-01A,9.869982,11.189493,9.633288,9.057744,9.087363,12.426148,11.915674,9.696256,10.315499,7.395802,...,9.323656,7.077178,8.490880,8.804611,8.738330,9.235010,8.087203,7.662738,10.633918,1
TCGA-3B-A9HL-01A,11.164872,10.429709,9.187715,8.614128,7.966475,10.053141,12.453439,9.209995,9.841264,8.130650,...,9.570093,6.574077,8.740703,6.786439,8.330560,8.966090,8.115032,7.395187,10.510393,1
TCGA-3B-A9HP-01A,9.614155,10.165624,9.562561,8.163207,8.471740,10.789869,11.155625,8.836156,9.954284,7.784037,...,9.668858,9.946461,9.038540,11.954611,8.698204,9.541148,8.040706,7.733620,10.819688,1


In [7]:
#####################################!#################################
#here begins full data
################################
#full data, 4 labels analysis
#Complete Response    21
#Clinical Progressive Disease    10
#Radiographic Progressive Disease     7
#Stable Disease     7

#features
df_raw_coln = len(df_raw.columns)
X = df_raw.iloc[:,0:(df_raw_coln-1)]
X = X.values

#label/target
y = df_raw.loc[:, 'response_group']
y = y.values

#!!!!!!!
#check to confirm the last column is not response group, only y contains response group information
col = X.shape[1]
print(X[:,(col-1)])

#df_cancer.head(10)
#df_normal.head(10)

class_names = np.unique(y)
print("unique labels from y: ", class_names)

[10.6339179  10.51039256 10.81968777 11.88574742 11.00516431 11.02850842
 10.60827299 11.01014098 10.36913487 10.0889746  11.04541671 10.72149906
 10.27210006 11.20329944 10.21014971 12.18407436 10.89397794 10.98949722
 10.42837596  9.25669936 11.14160236 12.07928841 11.00323572 12.05058529
 11.19723929 10.18340007  8.72905485 11.29278591 11.0301676  11.27654727
 11.21327947  9.80665461 11.1429714  11.12048614 11.65617548 10.72020451
 11.87490585  9.58506744 11.27991018 11.81112685 11.20157657 10.94837939
 12.14635861 12.07135191 10.94792596 10.02192759 10.50815203 11.23732727
 10.51992972 11.59623984 10.02183291 11.30336404  9.9603765  11.59921087
  9.53360723 11.34417309 10.95586297 10.83116231 11.49436733 11.00323513
 10.8359843  12.04624651 10.92790174 11.16791676 10.55554478]
unique labels from y:  [0 1]


In [9]:
########################################################
#importing necessary libraries for scikit-learn

from sklearn import svm, datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.ensemble import BaggingClassifier

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
#from sklearn.grid_search import GridSearchCV   #!!!the grid search package that has issue, dont use it
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from scipy import interp
from scipy import stats

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing

/home/qiwei/anaconda2/envs/tensorflow-gpu-test/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/home/qiwei/anaconda2/envs/tensorflow-gpu-test/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
#########################################################################################
#plot confusion matrix
#inputs: cm, confusion matrix from cross_val_predict
#        normalize, whether to use normalize for each sections 
#        title, input the title name for the figure
#        cmap, color map using blue as default
#output: a confusion matrix plot with true label as y axis, and predicted label as x axis
#########################################################################################
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [11]:
##############################################################
#plot area under curve graph
#input: actual, true labels/target without one hot encoding
#       probs, predicted probabilities
#       n_classes, number of unique classes in target
#       title, input the title name for the figure
#output: a roc curve plot for multi class task
###############################################################
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import LabelBinarizer
from itertools import cycle

def plot_multiclass_roc_auc(actual, probs, n_classes, title = 'multi-class roc'):
    lb = LabelBinarizer()
    lb.fit(actual)
    actual = lb.transform(actual)
    y_prob = probs
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(actual[:, i], y_prob[:, i])
        #please notice the difference between auc() and roc_auc_score()
        #also auc() only works on monotonic increasing or monotonic
        #decreasing input x
        roc_auc[i] = auc(fpr[i], tpr[i])
        
        colors = cycle(['blue', 'red', 'green', 'orange'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color,
        label='ROC curve of class {0} (area = {1:0.10f})'
            ''.format(i, roc_auc[i]))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([-0.05, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic for multi-class data using '+title)
    plt.legend(loc="lower right")
    #commented thus being able to use fig save function
    #plt.show()

In [12]:
#######################################################
#Random search CV method
#and
#Multi class roc_auc score method
########################################################
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from time import time
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import make_scorer, roc_auc_score

###########################################################################################
#Multi class roc_auc score method
#input: y_test, true labels from test fold
#       y_prob, predicted probability on test fold
#       average, string, [None, ‘micro’, ‘macro’ (default), ‘samples’, ‘weighted’]
#                'macro': Calculate metrics for each label, and find their unweighted mean. 
#                This does not take label imbalance into account.
#                'weighted': Calculate metrics for each label, and find their average, 
#                weighted by support (the number of true instances for each label).
#output: auroc value for each class
#multiclass_score, an implemented scoring method for multi class task
#!!!
#Notice that by default,needs_proba : boolean, default=False
#thus the multiclass_score will try to use the predicted label instead of predicted probability to calculate roc
#which is not correct, and will causing the tuning process to not find the best parameters
##############################################################################################
def multiclass_roc_auc_score(y_test, y_prob, average="weighted"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    #y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_prob, average=average)

#!!!
#Notice that by default,needs_proba : boolean, default=False
#thus the multiclass_score will try to use the predicted label instead of predicted probability to calculate roc
#which is not correct, and will causing the tuning process to not find the best parameters
multiclass_score = make_scorer(multiclass_roc_auc_score, needs_proba = True)


###############################################################################################
#Binary class roc auc score method
#input: y_true, true labels from test fold
#       y_score, predicted probability on test fold
#       average, string, [None, ‘micro’, ‘macro’ (default), ‘samples’, ‘weighted’]
#                'macro': Calculate metrics for each label, and find their unweighted mean. 
#                This does not take label imbalance into account.
#                'weighted': Calculate metrics for each label, and find their average, 
#                weighted by support (the number of true instances for each label).
#output: auroc value for each class
#############################################################################################
def binary_class_roc_auc_score(y_true, y_score, average="weighted"):

    return roc_auc_score(y_true, y_score, average=average)

binaryclass_score = make_scorer(binary_class_roc_auc_score, needs_threshold = True)

###################################################################################
#Random search CV method
#input: est, input estimator/classifier
#       p_distr, the grid of parameters to search on
#       nbr_iter, numbers of iteration on random search
#       X, feature, y, true labels
#output: ht_estimator, best estimator based on mean value of all folds
#        ht_params, best parameters
#
################################################################################################
def hypertuning_rscv(est, p_distr, nbr_iter,X,y):
    #seed = 42
    cv = StratifiedKFold(n_splits = 3, random_state = r_seed, shuffle = True)
    rdmsearch = RandomizedSearchCV(est, param_distributions=p_distr, scoring=multiclass_score,
                                  n_jobs=-1, n_iter=nbr_iter, cv=cv, return_train_score = True, verbose =10)
    #CV = Cross-Validation ( here using Stratified KFold CV) #,random_state = seed
    start = time()
    rdmsearch.fit(X,y)
    print('hyper-tuning time : %d seconds' % (time()-start))
    start = 0
   # ht_train_mean = rdmsearch.cv_results_['mean_train_score']
   # ht_train_std = rdmsearch.cv_results_['std_train_score']
   # ht_test_mean_sp0 = rdmsearch.cv_results_['split0_test_score']
   # ht_test_mean_sp1 = rdmsearch.cv_results_['split1_test_score']
   # ht_test_mean_sp2 = rdmsearch.cv_results_['split2_test_score']
    #ht_train_mean_sp3 = rdmsearch.cv_results_['split3_train_score']
    #ht_train_mean_sp4 = rdmsearch.cv_results_['split4_train_score']
    #ht_best_loc = np.where(rdmsearch.cv_results_['rank_test_score'] == 1)
    
    ht_cv_results = rdmsearch.cv_results_
    ht_estimator = rdmsearch.best_estimator_
    ht_params = rdmsearch.best_params_
    #ht_score = rdmsearch.best_score_
    
    return ht_estimator, ht_params, ht_cv_results

In [13]:
##########################################################################
#just backup pipeline to train a default knn estimator
#test to confirm whether is imbalanced number in folds changes result
###############################################################################
#KNN
# dividing X, y into train and test data
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/3, random_state = 42)
 
# training a KNN classifier
#from sklearn.neighbors import KNeighborsClassifier
#knn = KNeighborsClassifier(n_neighbors = 6, p = 3, weights = "distance", algorithm = "ball_tree", 
#                          leaf_size = 18).fit(X_train, y_train)

#accuracy on X_test
#accuracy = knn.score(X_test, y_test)
#print (accuracy)

# return predict probability
#knn_prob = knn.predict_proba(X_test)
#print(knn_prob)

# calculate the auroc by directly using the multiclass_roc_auc_score scorer
#knn_multiclass_auroc = multiclass_roc_auc_score(y_test, knn_prob, average="macro")
#print(knn_multiclass_auroc)

# calculate the individual auroc and then calculate the mean by plotting on figure
#import matplotlib.pyplot as plt
#plt.figure(figsize = (10, 8))
#plot_multiclass_roc_auc(y_test, knn_prob, n_classes = 4, title = "test, knn")
#plt.savefig('test.png')

#print((0.53704+0.69318+0.72222+1)/4)
# creating a confusion matrix
#knn_predictions = knn.predict(X_test) 
#cm = confusion_matrix(y_test, knn_predictions)

# training a KNN classifier
#from sklearn.neighbors import KNeighborsClassifier

#knn = KNeighborsClassifier(n_neighbors = 7)
#knn_scores = cross_val_score(knn, X, y, cv = 5)
#print(knn)
#print("Accuracy: %0.5f (+/- %0.4f)" % (knn_scores.mean(), stats.sem(knn_scores) * 2))

#from sklearn.model_selection import cross_val_predict
#from sklearn.metrics import confusion_matrix
#y_knn_pred = cross_val_predict(knn, X, y, cv = 3)
#knn_conf_mat = confusion_matrix(y,y_knn_pred)
#print(knn_conf_mat)

#import matplotlib.pyplot as plt
#import itertools

#plt.figure(figsize = (14, 14))
#plot_confusion_matrix(knn_conf_mat, classes=class_names,
#                      title='Confusion matrix knn, without normalization')

In [14]:
#############################################
#Grid search Tune LNN
#default, without bagging and Ensemble
###################################################
#from sklearn.neighbors import KNeighborsClassifier
#param_test1 = {
#            'n_neighbors':[i for i in range(1,20)],
#            'weights':['uniform', 'distance'],
#            'algorithm':['ball_tree', 'kd_tree','brute','auto'],
#            'leaf_size':[i for i in range(1,20)],
#            'p':[i for i in range(1,3)]
#}
#cv = StratifiedKFold(n_splits = 3, random_state = 42)

#gsearch1 = GridSearchCV(estimator = KNeighborsClassifier(), 
#param_grid = param_test1, scoring=multiclass_score,n_jobs=-1,iid=False, cv=cv, return_train_score=True)
#gsearch1.fit(X,y)
#gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [15]:
#does have overfitting issue
#print(gsearch1.cv_results_["split1_train_score"])
#print(gsearch1.cv_results_["split1_test_score"])

In [16]:
#############################################
#Tuning KNN with Grid search
#output: best estimator and best parameters
#default, with bagging and Ensemble
###################################################
#param_test1 = {
#            'n_neighbors':[i for i in range(1,20)],
#            'weights':['uniform', 'distance'],
#            'algorithm':['ball_tree', 'kd_tree','brute','auto'],
#            'leaf_size':[i for i in range(1,20)],
#            'p':[i for i in range(1,4)]
#}
#cv = StratifiedKFold(n_splits = 3, random_state = 42, shuffle = True)

#gsearch1 = GridSearchCV(estimator = KNeighborsClassifier(), 
#param_grid = param_test1, scoring=multiclass_score,n_jobs=-1,iid=False, cv=cv, return_train_score=True, verbose = 10)
#gsearch1.fit(X,y)
#gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [17]:
#does have overfitting issue
#print(gsearch1.cv_results_["split1_train_score"])
#print(gsearch1.cv_results_["split1_test_score"])

In [18]:
#########################################################################################
#Tuning KNN with randomsearch
#output: best estimator and best parameters
#!!!
##Notice there is issue with the best_score_ output from the randomSearchCV, dont use it
#############################################################################################
#from sklearn.neighbors import KNeighborsClassifier
#est = KNeighborsClassifier()

#knn_p_dist={'n_neighbors':[i for i in range(1,20)],
#            'weights':['uniform', 'distance'],
#            'algorithm':['ball_tree', 'kd_tree','brute','auto'],
#            'leaf_size':[i for i in range(1,20)],
#            'p':[i for i in range(1,5)]
#           }

#knn_estimator, knn_parameters, knn_cv_results = hypertuning_rscv(est, knn_p_dist, 800, X, y)
#print(knn_parameters)

#!!!
#Notice there is issue with the best_score_ output from the randomSearchCV
#print('Hyper-tuned model score :')
#print(knn_ht_score*100)

In [19]:
#print(knn_cv_results["split1_train_score"])
#print(knn_cv_results["split1_test_score"])

In [20]:
#########################################################################
#training a KNN classifier use best estimator found by random search
######################################################################
#from sklearn.neighbors import KNeighborsClassifier
#knn = KNeighborsClassifier(n_neighbors = 6, weights = 'distance', leaf_size =1, algorithm = 'ball_tree', 
#                           p=2)

# if using the randomSearch method
#knn = knn_estimator

# if using GridSearch method
#knn = gsearch1.best_estimator_
#cv = StratifiedKFold(n_splits = 3, random_state = 42, shuffle = True)

##!!!!
#notice that mean of auroc of each fold is different from the auroc calculated by all the predicted probability
#knn_scores = cross_val_score(knn, X, y, cv = cv, scoring=multiclass_score)
#y_knn_prob = cross_val_predict(knn, X, y, cv = cv, method = 'predict_proba')

# calculate the auroc by directly using the multiclass_roc_auc_score scorer
#knn_multiclass_auroc = multiclass_roc_auc_score(y, y_knn_prob, average="weighted")

#print(knn)
#print("Auroc across all folds: %0.5f" % (knn_multiclass_auroc))

In [21]:
########################################
#print out confusion matrix
###############################################

#from sklearn.model_selection import cross_val_predict
#from sklearn.metrics import confusion_matrix

#cv = StratifiedKFold(n_splits = 3, random_state = 42, shuffle = True)
#y_knn_pred = cross_val_predict(knn, X, y, cv = cv)
#knn_conf_mat = confusion_matrix(y,y_knn_pred)

#from sklearn.metrics import roc_auc_score
#print(roc_auc_score(y, y_knn_pred))

#print(knn_conf_mat)

In [22]:
########################################
#print out multiclass roc auc figure
###############################################
#from sklearn.model_selection import cross_val_predict
#from sklearn.metrics import confusion_matrix

#cv = StratifiedKFold(n_splits = 3, random_state = 42)
#y_knn_prob = cross_val_predict(knn, X, y, cv = cv, method = 'predict_proba')

#print(y_knn_prob)

#import matplotlib.pyplot as plt
#plt.figure(figsize = (10, 8))
#plot_multiclass_roc_auc(y, y_knn_prob, n_classes = 3, title = "knn, SARC0.2")
#plt.savefig('result/3_classes/(0606)Multiclass_roc_auc_knn(SARC0.2).png')

In [23]:
##################################################################
#Generate confusion matrix figure and save those figures
#Notice remember to change file name based on the input dataset
###################################################################
#import matplotlib.pyplot as plt
#import itertools
#from matplotlib.pyplot import legend

#labels = ['Complete Response', 'Clinical Progressive Disease', 'Radiographic Progressive Disease', 'Stable Disease']

#plt.figure(figsize = (20, 15))
#legend(labels)
#plot_confusion_matrix(knn_conf_mat, classes=labels,
#                      title='Confusion matrix knn, without normalization')
#plt.savefig('result/3_classes/(0414)Confusion_matrix_knn(SARC0.1)(without_normalization).png')


#plt.figure(figsize = (20, 15))
#legend(labels)
#plot_confusion_matrix(knn_conf_mat, classes=labels, normalize = True,
#                      title='Normalized Confusion matrix knn')
#plt.savefig('result/3_classes/(0414)Confusion_matrix_knn(SARC0.1)(normalized).png')

In [24]:
##########################################################################
#just backup pipeline to train a defaultlinear SVM classifier
#test to confirm whether is imbalanced number in folds changes result
###############################################################################

#from sklearn.svm import SVC
#svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train)
#svm_predictions = svm_model_linear.predict(X_test)
 
# model accuracy for X_test  
#accuracy = svm_model_linear.score(X_test, y_test)
#print(accuracy)

# creating a confusion matrix
#cm = confusion_matrix(y_test, svm_predictions)

#from sklearn.svm import SVC
#svm_model_linear = SVC(kernel = 'linear', C = 1)
#svm_scores = cross_val_score(svm_model_linear, X, y, cv = 5)
#print(svm_model_linear)
#print("Accuracy: %0.4f (+/- %0.4f)" % (svm_scores.mean(), stats.sem(svm_scores) * 2))

#from sklearn.model_selection import cross_val_predict
#from sklearn.metrics import confusion_matrix
#y_svm_pred = cross_val_predict(svm_model_linear, X, y, cv = 10)
#svm_conf_mat = confusion_matrix(y,y_svm_pred)
#print(svm_conf_mat)

#import matplotlib.pyplot as plt
#import itertools

#plt.figure(figsize = (14, 14))
#plot_confusion_matrix(svm_conf_mat, classes=class_names,
#                      title='Confusion matrix svm, without normalization')


In [25]:
#############################################
#Tuning SVM with Grid search
#output: best estimator and best parameters
#default, with bagging and Ensemble
###################################################
#param_test2 = {
#            'kernel':['linear','poly','rbf','sigmoid'],
#            'C':[i for i in range(5, 50)],
#            'gamma':['auto'],
#            'degree':[i for i in range(1,20)],
#            'probability':[True],
#            #'max_iter':[2000],
#            'random_state':[42]
#}
#cv = StratifiedKFold(n_splits = 3, random_state = 42, shuffle = True)

#gsearch2 = GridSearchCV(estimator = SVC(), 
#param_grid = param_test2, scoring=multiclass_score,n_jobs=-1,iid=False, cv=cv, return_train_score=True, verbose =10)
#gsearch2.fit(X,y)
#gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

In [26]:
#Notice that SVM need the input X to be scaled to [-1, 1] range
#therefore using Standarization to transform X first
# Standardize
#scaler = preprocessing.StandardScaler()
#scaler.fit((X))
#X_std = scaler.transform(X)
#print(X_std.shape)

In [27]:
#################################
#Tuning SVM with randomsearch
########################################
#from sklearn.svm import SVC
#est = SVC()
#from scipy.stats import norm
#svc_p_dist={'kernel':['linear','poly','rbf','sigmoid'],
#            'C':[i/10 for i in range (1,1000)],
#            'gamma':['auto'],
#            'degree':[i for i in range(1,50)],
#            'probability':[True],
#            #'max_iter':[2000],
#            'random_state':[42]}

#svc_estimator, svc_parameters, svc_cv_results = hypertuning_rscv(est, svc_p_dist, 700, X_std, y)

#print(svc_parameters)

#print('Hyper-tuned model score :')
#print(svc_ht_score*100)


In [28]:
#print(svc_cv_results["split1_train_score"])
#print(svc_cv_results["split1_test_score"])

In [29]:
#from sklearn.model_selection import cross_val_predict
#from sklearn.svm import SVC
#svm_model_linear = SVC(kernel = 'linear', C = 0.4861735698828815, probability = True)

# if using the randomSearch method
#svm_model_linear = svc_estimator

# if using GridSearch method
#svm_model_linear = gsearch2.best_estimator_

#cv = StratifiedKFold(n_splits = 3, random_state = 42, shuffle = True)

##!!!!
#notice that mean of auroc of each fold is different from the auroc calculated by all the predicted probability
#svm_scores = cross_val_score(svm_model_linear, X, y, cv = cv, scoring=multiclass_score)
#y_svm_prob = cross_val_predict(svm_model_linear, X, y, cv = cv, method = 'predict_proba')

# calculate the auroc by directly using the multiclass_roc_auc_score scorer
#svm_multiclass_auroc = multiclass_roc_auc_score(y, y_svm_prob, average="weighted")

#print(svm_model_linear)
#print("Auroc across all folds: %0.5f" % (svm_multiclass_auroc))

In [30]:
#from sklearn.model_selection import cross_val_predict
#from sklearn.metrics import confusion_matrix

#cv = StratifiedKFold(n_splits = 3, random_state = 42, shuffle = True)
#y_svm_pred = cross_val_predict(svm_model_linear, X, y, cv = cv)
#svm_conf_mat = confusion_matrix(y,y_svm_pred)

#from sklearn.metrics import roc_auc_score
#print(roc_auc_score(y, y_svm_pred))

#print(svm_conf_mat)

In [31]:
#(0.46627+0.52571+0.46992+0.93233)/4

In [32]:
########################################
#print out multiclass roc auc figure
############################################
#from sklearn.metrics import confusion_matrix

#cv = StratifiedKFold(n_splits = 3, random_state = 42)
#y_svm_prob = cross_val_predict(svm_model_linear, X, y, cv = cv, method = 'predict_proba')

#import matplotlib.pyplot as plt
#plt.figure(figsize = (10, 8))
#plot_multiclass_roc_auc(y, y_svm_prob, n_classes = 3, title = "svm, SARC0.2")
#plt.savefig('result/3_classes/(0606)Multiclass_roc_auc_svm(SARC0.2).png')

In [33]:
##################################################################
#Generate confusion matrix figure and save those figures
#Notice remember to change file name based on the input dataset
###################################################################
#import matplotlib.pyplot as plt
#import itertools
#from matplotlib.pyplot import legend

#labels = ['Complete Response', 'Clinical Progressive Disease', 'Radiographic Progressive Disease', 'Stable Disease']

#plt.figure(figsize = (20, 15))
#legend(labels)
#plot_confusion_matrix(svm_conf_mat, classes=labels,
#                      title='Confusion matrix svm, without normalization')
#plt.savefig('result/3_classes/(0414)Confusion_matrix_svm(SARC0.1)(without_normalization).png')


#plt.figure(figsize = (20, 15))
#legend(labels)
#plot_confusion_matrix(svm_conf_mat, classes=labels, normalize = True,
#                      title='Normalized Confusion matrix svm')
#plt.savefig('result/3_classes/(0414)Confusion_matrix_svm(SARC0.1)(normalized).png')

In [34]:
##########################################################################
#just backup pipeline to train a XGBoost model
#test to confirm whether is imbalanced number in folds changes result
###############################################################################
#xgb = XGBClassifier(
#                    learning_rate =0.1,
#                    n_estimators=1000,
#                    max_depth=5,
#                     min_child_weight=1,
#                     gamma=0,
#                     subsample=0.8,
#                     colsample_bytree=0.8,
#                     objective= 'binary:logistic',
#                     scale_pos_weight=1,
#                    seed=42)

#xgb_scores = cross_val_score(xgb, X, y, cv = 3)
#print(xgb)
#print("Accuracy: %0.5f (+/- %0.4f)" % (xgb_scores.mean(), stats.sem(xgb_scores) * 2))

In [35]:
#############################################
#Tuning XGboost with Grid search
#output: best estimator and best parameters
#default
###################################################
#param_test3 = {
#            'learning_rate':[0.4, 0.5, 0.6],
#            'n_estimators':[i for i in range(12,20)],
            #'n_estimators':[i for i in range(1,10)],
#            'max_depth':[i for i in range(12,20)],
#             'min_child_weight':[i for i in range(0,10)],
#             'booster':['gbtree','gblinear','dart'],
#             'subsample':[i/100.0 for i in range(70,90)],
#             'colsample_bytree':[i/100.0 for i in range(40,65)],
#             'reg_alpha':[i for i in range(0,5)],
#             'reg_lambda':[i for i in range(100,120)],
#             'silent':[True],
#             'random_state':[42]
#}
#cv = StratifiedKFold(n_splits = 3, random_state = 42)

#gsearch3 = GridSearchCV(estimator = XGBClassifier(), 
#param_grid = param_test3, scoring=multiclass_score,n_jobs=-1,iid=False, cv=cv, return_train_score=True,verbose=10)
#gsearch3.fit(X,y)
#gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

In [36]:
###########################################################
#Grid search Tune learning rate, n_estimators, and booster
#default is max_depth = 5, min_child_weight = 1
###################################################
param_test3 = {
 'learning_rate':[0.05, 0.1, 0.2, 0.4, 0.6, 0.8],
 'n_estimators':[i for i in range(1,40)],
 'booster':['gbtree'],
    'silent':[True],
    'random_state':[r_seed]
}
cv = StratifiedKFold(n_splits = num_cv, random_state = r_seed, shuffle = True)

gsearch3 = GridSearchCV(estimator = XGBClassifier(), 
 param_grid = param_test3, scoring="roc_auc",n_jobs=-1,iid=False, cv=cv,verbose=10)
gsearch3.fit(X,y)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

Fitting 5 folds for each of 234 candidates, totalling 1170 fits
[CV] random_state=55555, booster=gbtree, n_estimators=1, silent=True, learning_rate=0.05 
[CV] random_state=55555, booster=gbtree, n_estimators=1, silent=True, learning_rate=0.05 
[CV] random_state=55555, booster=gbtree, n_estimators=1, silent=True, learning_rate=0.05 
[CV] random_state=55555, booster=gbtree, n_estimators=1, silent=True, learning_rate=0.05 
[CV] random_state=55555, booster=gbtree, n_estimators=1, silent=True, learning_rate=0.05 
[CV] random_state=55555, booster=gbtree, n_estimators=2, silent=True, learning_rate=0.05 
[CV] random_state=55555, booster=gbtree, n_estimators=2, silent=True, learning_rate=0.05 
[CV] random_state=55555, booster=gbtree, n_estimators=2, silent=True, learning_rate=0.05 
[CV]  random_state=55555, booster=gbtree, n_estimators=1, silent=True, learning_rate=0.05, score=0.5476190476190477, total=   0.1s
[CV] random_state=55555, booster=gbtree, n_estimators=2, silent=True, learning_rate=0

[Parallel(n_jobs=-1)]: Batch computation too fast (0.1717s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.2s


[CV]  random_state=55555, booster=gbtree, n_estimators=2, silent=True, learning_rate=0.05, score=0.523809523809524, total=   0.2s
[CV] random_state=55555, booster=gbtree, n_estimators=4, silent=True, learning_rate=0.05 
[CV]  random_state=55555, booster=gbtree, n_estimators=2, silent=True, learning_rate=0.05, score=0.6785714285714286, total=   0.3s
[CV] random_state=55555, booster=gbtree, n_estimators=4, silent=True, learning_rate=0.05 
[CV]  random_state=55555, booster=gbtree, n_estimators=3, silent=True, learning_rate=0.05, score=0.3020833333333333, total=   0.3s
[CV] random_state=55555, booster=gbtree, n_estimators=4, silent=True, learning_rate=0.05 
[CV]  random_state=55555, booster=gbtree, n_estimators=2, silent=True, learning_rate=0.05, score=0.6142857142857143, total=   0.3s
[CV]  random_state=55555, booster=gbtree, n_estimators=3, silent=True, learning_rate=0.05, score=0.5833333333333334, total=   0.3s
[CV] random_state=55555, booster=gbtree, n_estimators=5, silent=True, learni

[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.5s


[CV]  random_state=55555, booster=gbtree, n_estimators=3, silent=True, learning_rate=0.05, score=0.6428571428571429, total=   0.3s
[CV] random_state=55555, booster=gbtree, n_estimators=6, silent=True, learning_rate=0.05 
[CV]  random_state=55555, booster=gbtree, n_estimators=3, silent=True, learning_rate=0.05, score=0.6785714285714286, total=   0.3s
[CV] random_state=55555, booster=gbtree, n_estimators=6, silent=True, learning_rate=0.05 
[CV]  random_state=55555, booster=gbtree, n_estimators=4, silent=True, learning_rate=0.05, score=0.6666666666666666, total=   0.3s
[CV] random_state=55555, booster=gbtree, n_estimators=4, silent=True, learning_rate=0.05 
[CV]  random_state=55555, booster=gbtree, n_estimators=4, silent=True, learning_rate=0.05, score=0.3020833333333333, total=   0.4s
[CV] random_state=55555, booster=gbtree, n_estimators=7, silent=True, learning_rate=0.05 
[CV]  random_state=55555, booster=gbtree, n_estimators=4, silent=True, learning_rate=0.05, score=0.6428571428571429,

...

[Parallel(n_jobs=-1)]: Done 1119 tasks      | elapsed:  2.1min


[CV]  random_state=55555, booster=gbtree, n_estimators=30, silent=True, learning_rate=0.8, score=0.07291666666666666, total=   0.8s
[CV] random_state=55555, booster=gbtree, n_estimators=31, silent=True, learning_rate=0.8 
[CV]  random_state=55555, booster=gbtree, n_estimators=30, silent=True, learning_rate=0.8, score=0.6190476190476191, total=   0.7s
[CV] random_state=55555, booster=gbtree, n_estimators=32, silent=True, learning_rate=0.8 
[CV]  random_state=55555, booster=gbtree, n_estimators=30, silent=True, learning_rate=0.8, score=0.4761904761904762, total=   0.6s
[CV] random_state=55555, booster=gbtree, n_estimators=32, silent=True, learning_rate=0.8 
[CV]  random_state=55555, booster=gbtree, n_estimators=30, silent=True, learning_rate=0.8, score=0.5428571428571428, total=   0.7s
[CV] random_state=55555, booster=gbtree, n_estimators=32, silent=True, learning_rate=0.8 
[CV]  random_state=55555, booster=gbtree, n_estimators=31, silent=True, learning_rate=0.8, score=0.0729166666666666

[CV]  random_state=55555, booster=gbtree, n_estimators=38, silent=True, learning_rate=0.8, score=0.07291666666666666, total=   0.7s
[CV] random_state=55555, booster=gbtree, n_estimators=39, silent=True, learning_rate=0.8 
[CV] random_state=55555, booster=gbtree, n_estimators=39, silent=True, learning_rate=0.8 
[CV]  random_state=55555, booster=gbtree, n_estimators=37, silent=True, learning_rate=0.8, score=0.6190476190476191, total=   0.9s
[CV] random_state=55555, booster=gbtree, n_estimators=39, silent=True, learning_rate=0.8 
[CV]  random_state=55555, booster=gbtree, n_estimators=38, silent=True, learning_rate=0.8, score=0.6904761904761906, total=   0.7s
[CV] random_state=55555, booster=gbtree, n_estimators=39, silent=True, learning_rate=0.8 
[CV]  random_state=55555, booster=gbtree, n_estimators=38, silent=True, learning_rate=0.8, score=0.4761904761904762, total=   0.7s
[CV]  random_state=55555, booster=gbtree, n_estimators=38, silent=True, learning_rate=0.8, score=0.6190476190476191

[Parallel(n_jobs=-1)]: Done 1170 out of 1170 | elapsed:  2.2min finished
/home/qiwei/anaconda2/envs/tensorflow-gpu-test/lib/python3.5/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.53565, std: 0.13038, params: {'random_state': 55555, 'booster': 'gbtree', 'silent': True, 'n_estimators': 1, 'learning_rate': 0.05},
  mean: 0.54280, std: 0.13008, params: {'random_state': 55555, 'booster': 'gbtree', 'silent': True, 'n_estimators': 2, 'learning_rate': 0.05},
  mean: 0.53661, std: 0.13585, params: {'random_state': 55555, 'booster': 'gbtree', 'silent': True, 'n_estimators': 3, 'learning_rate': 0.05},
  mean: 0.55042, std: 0.13514, params: {'random_state': 55555, 'booster': 'gbtree', 'silent': True, 'n_estimators': 4, 'learning_rate': 0.05},
  mean: 0.54185, std: 0.12320, params: {'random_state': 55555, 'booster': 'gbtree', 'silent': True, 'n_estimators': 5, 'learning_rate': 0.05},
  mean: 0.53470, std: 0.11734, params: {'random_state': 55555, 'booster': 'gbtree', 'silent': True, 'n_estimators': 6, 'learning_rate': 0.05},
  mean: 0.54994, std: 0.12537, params: {'random_state': 55555, 'booster': 'gbtree', 'silent': True, 'n_estimators': 7, 'learning_rate': 0.05},

In [ ]:
##########################################################
#output grid scores, and save to a file
#
#xgb_grid_scores = pd.DataFrame(gsearch1.grid_scores_)
#xgb_grid_file = os.path.join("Tuning_insights", "xgb_grid_socres(lr&n_estimators).tsv")
#xgb_grid_scores.to_csv(xgb_grid_file, sep='\t')

In [37]:
#################################################
#Grid search Tune max_depth and min_child_weight
#default
#######################################
param_test3 = {
 'max_depth':[i for i in range(1,10)],
 'min_child_weight':[i for i in range(0,10)],
    'silent':[True],
    'random_state':[r_seed]
}
cv = StratifiedKFold(n_splits = num_cv, random_state = r_seed, shuffle = True)

gsearch3 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=23, booster = 'gbtree'), 
 param_grid = param_test3, scoring="roc_auc",n_jobs=-1,iid=False, cv=cv,verbose=10)
gsearch3.fit(X,y)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV] random_state=55555, silent=True, max_depth=1, min_child_weight=0 
[CV] random_state=55555, silent=True, max_depth=1, min_child_weight=0 
[CV] random_state=55555, silent=True, max_depth=1, min_child_weight=0 
[CV] random_state=55555, silent=True, max_depth=1, min_child_weight=0 
[CV] random_state=55555, silent=True, max_depth=1, min_child_weight=0 
[CV] random_state=55555, silent=True, max_depth=1, min_child_weight=1 
[CV] random_state=55555, silent=True, max_depth=1, min_child_weight=1 
[CV] random_state=55555, silent=True, max_depth=1, min_child_weight=1 
[CV]  random_state=55555, silent=True, max_depth=1, min_child_weight=1, score=0.5208333333333333, total=   0.5s
[CV]  random_state=55555, silent=True, max_depth=1, min_child_weight=0, score=0.6190476190476191, total=   0.6s
[CV] random_state=55555, silent=True, max_depth=1, min_child_weight=1 
[CV]  random_state=55555, silent=True, max_depth=1, min_child_weight=0, sco

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.7s


[CV]  random_state=55555, silent=True, max_depth=1, min_child_weight=1, score=0.5714285714285714, total=   0.8s
[CV] random_state=55555, silent=True, max_depth=1, min_child_weight=3 
[CV]  random_state=55555, silent=True, max_depth=1, min_child_weight=1, score=0.30952380952380953, total=   0.5s
[CV] random_state=55555, silent=True, max_depth=1, min_child_weight=3 
[CV]  random_state=55555, silent=True, max_depth=1, min_child_weight=2, score=0.5208333333333333, total=   0.6s
[CV]  random_state=55555, silent=True, max_depth=1, min_child_weight=1, score=0.6571428571428571, total=   0.6s
[CV] random_state=55555, silent=True, max_depth=1, min_child_weight=3 
[CV]  random_state=55555, silent=True, max_depth=1, min_child_weight=2, score=0.5714285714285714, total=   0.5s
[CV] random_state=55555, silent=True, max_depth=1, min_child_weight=3 
[CV]  random_state=55555, silent=True, max_depth=1, min_child_weight=2, score=0.6190476190476191, total=   0.6s
[CV] random_state=55555, silent=True, max_d

[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.5s


[CV] random_state=55555, silent=True, max_depth=1, min_child_weight=4 
[CV]  random_state=55555, silent=True, max_depth=1, min_child_weight=3, score=0.5952380952380952, total=   0.5s
[CV] random_state=55555, silent=True, max_depth=1, min_child_weight=4 
[CV]  random_state=55555, silent=True, max_depth=1, min_child_weight=3, score=0.6190476190476191, total=   0.5s
[CV]  random_state=55555, silent=True, max_depth=1, min_child_weight=4, score=0.375, total=   0.4s
[CV] random_state=55555, silent=True, max_depth=1, min_child_weight=5 
[CV]  random_state=55555, silent=True, max_depth=1, min_child_weight=3, score=0.33333333333333337, total=   0.5s
[CV] random_state=55555, silent=True, max_depth=1, min_child_weight=5 
[CV] random_state=55555, silent=True, max_depth=1, min_child_weight=5 
[CV]  random_state=55555, silent=True, max_depth=1, min_child_weight=4, score=0.6904761904761904, total=   0.4s
[CV]  random_state=55555, silent=True, max_depth=1, min_child_weight=4, score=0.5714285714285714,

...

[CV]  random_state=55555, silent=True, max_depth=9, min_child_weight=0, score=0.6857142857142857, total=   2.1s
[CV] random_state=55555, silent=True, max_depth=9, min_child_weight=3 
[CV]  random_state=55555, silent=True, max_depth=9, min_child_weight=1, score=0.5238095238095238, total=   1.3s
[CV] random_state=55555, silent=True, max_depth=9, min_child_weight=3 
[CV]  random_state=55555, silent=True, max_depth=9, min_child_weight=2, score=0.45833333333333337, total=   1.0s
[CV] random_state=55555, silent=True, max_depth=9, min_child_weight=3 
[CV]  random_state=55555, silent=True, max_depth=9, min_child_weight=2, score=0.7142857142857143, total=   1.0s
[CV] random_state=55555, silent=True, max_depth=9, min_child_weight=3 
[CV]  random_state=55555, silent=True, max_depth=9, min_child_weight=2, score=0.5952380952380952, total=   1.0s
[CV] random_state=55555, silent=True, max_depth=9, min_child_weight=3 
[CV]  random_state=55555, silent=True, max_depth=9, min_child_weight=3, score=0.5416

[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   43.1s


[CV]  random_state=55555, silent=True, max_depth=9, min_child_weight=8, score=0.5, total=   0.3s
[CV] random_state=55555, silent=True, max_depth=9, min_child_weight=9 
[CV]  random_state=55555, silent=True, max_depth=9, min_child_weight=8, score=0.5, total=   0.4s
[CV] random_state=55555, silent=True, max_depth=9, min_child_weight=9 
[CV]  random_state=55555, silent=True, max_depth=9, min_child_weight=8, score=0.5, total=   0.4s
[CV]  random_state=55555, silent=True, max_depth=9, min_child_weight=8, score=0.5, total=   0.3s
[CV]  random_state=55555, silent=True, max_depth=9, min_child_weight=8, score=0.5, total=   0.4s
[CV]  random_state=55555, silent=True, max_depth=9, min_child_weight=9, score=0.5, total=   0.3s
[CV]  random_state=55555, silent=True, max_depth=9, min_child_weight=9, score=0.5, total=   0.3s
[CV]  random_state=55555, silent=True, max_depth=9, min_child_weight=9, score=0.5, total=   0.3s
[CV]  random_state=55555, silent=True, max_depth=9, min_child_weight=9, score=0.5,

[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:   43.7s finished
/home/qiwei/anaconda2/envs/tensorflow-gpu-test/lib/python3.5/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.53560, std: 0.12194, params: {'random_state': 55555, 'silent': True, 'max_depth': 1, 'min_child_weight': 0},
  mean: 0.53560, std: 0.12194, params: {'random_state': 55555, 'silent': True, 'max_depth': 1, 'min_child_weight': 1},
  mean: 0.53560, std: 0.12194, params: {'random_state': 55555, 'silent': True, 'max_depth': 1, 'min_child_weight': 2},
  mean: 0.52274, std: 0.11736, params: {'random_state': 55555, 'silent': True, 'max_depth': 1, 'min_child_weight': 3},
  mean: 0.51976, std: 0.14097, params: {'random_state': 55555, 'silent': True, 'max_depth': 1, 'min_child_weight': 4},
  mean: 0.58083, std: 0.09632, params: {'random_state': 55555, 'silent': True, 'max_depth': 1, 'min_child_weight': 5},
  mean: 0.59315, std: 0.07195, params: {'random_state': 55555, 'silent': True, 'max_depth': 1, 'min_child_weight': 6},
  mean: 0.50000, std: 0.00000, params: {'random_state': 55555, 'silent': True, 'max_depth': 1, 'min_child_weight': 7},
  mean: 0.50000, std: 0.00000, params: {'random_

In [38]:
###########################
#Grid search Tune subsample and colsample
#
#################################
param_test3 = {
             'subsample':[i/100.0 for i in range(10,110,10)],
             'colsample_bytree':[i/100.0 for i in range(10,110,10)],
             
    'silent':[True],
    'random_state':[r_seed]
}
cv = StratifiedKFold(n_splits = num_cv, random_state = r_seed, shuffle = True)

gsearch3 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=23, booster = 'gbtree',
                                                 max_depth = 3, min_child_weight=1), 
  param_grid = param_test3, scoring="roc_auc",n_jobs=-1,iid=False, cv=cv,verbose=10)
gsearch3.fit(X,y)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] random_state=55555, subsample=0.1, colsample_bytree=0.1, silent=True 
[CV] random_state=55555, subsample=0.1, colsample_bytree=0.1, silent=True 
[CV] random_state=55555, subsample=0.1, colsample_bytree=0.1, silent=True 
[CV] random_state=55555, subsample=0.1, colsample_bytree=0.1, silent=True 
[CV] random_state=55555, subsample=0.1, colsample_bytree=0.1, silent=True 
[CV] random_state=55555, subsample=0.2, colsample_bytree=0.1, silent=True 
[CV] random_state=55555, subsample=0.2, colsample_bytree=0.1, silent=True 
[CV] random_state=55555, subsample=0.2, colsample_bytree=0.1, silent=True 
[CV]  random_state=55555, subsample=0.1, colsample_bytree=0.1, silent=True, score=0.6458333333333334, total=   0.1s
[CV] random_state=55555, subsample=0.2, colsample_bytree=0.1, silent=True 
[CV]  random_state=55555, subsample=0.1, colsample_bytree=0.1, silent=True, score=0.44047619047619047, total=   0.1s
[CV] random_state=55555, subs

[Parallel(n_jobs=-1)]: Batch computation too fast (0.1866s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.2s


[CV]  random_state=55555, subsample=0.2, colsample_bytree=0.1, silent=True, score=0.5428571428571429, total=   0.2s
[CV]  random_state=55555, subsample=0.2, colsample_bytree=0.1, silent=True, score=0.5476190476190477, total=   0.2s
[CV] random_state=55555, subsample=0.4, colsample_bytree=0.1, silent=True 
[CV] random_state=55555, subsample=0.4, colsample_bytree=0.1, silent=True 
[CV]  random_state=55555, subsample=0.2, colsample_bytree=0.1, silent=True, score=0.7142857142857142, total=   0.3s
[CV]  random_state=55555, subsample=0.3, colsample_bytree=0.1, silent=True, score=0.6190476190476191, total=   0.2s
[CV]  random_state=55555, subsample=0.3, colsample_bytree=0.1, silent=True, score=0.5208333333333334, total=   0.2s
[CV] random_state=55555, subsample=0.4, colsample_bytree=0.1, silent=True 
[CV] random_state=55555, subsample=0.5, colsample_bytree=0.1, silent=True 
[CV]  random_state=55555, subsample=0.3, colsample_bytree=0.1, silent=True, score=0.3809523809523809, total=   0.2s
[CV]

[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.4s


[CV]  random_state=55555, subsample=0.3, colsample_bytree=0.1, silent=True, score=0.5428571428571428, total=   0.2s
[CV]  random_state=55555, subsample=0.4, colsample_bytree=0.1, silent=True, score=0.4166666666666667, total=   0.2s
[CV] random_state=55555, subsample=0.6, colsample_bytree=0.1, silent=True 
[CV] random_state=55555, subsample=0.7, colsample_bytree=0.1, silent=True 
[CV]  random_state=55555, subsample=0.4, colsample_bytree=0.1, silent=True, score=0.7857142857142856, total=   0.2s
[CV] random_state=55555, subsample=0.4, colsample_bytree=0.1, silent=True 
[CV]  random_state=55555, subsample=0.5, colsample_bytree=0.1, silent=True, score=0.45238095238095233, total=   0.2s
[CV]  random_state=55555, subsample=0.5, colsample_bytree=0.1, silent=True, score=0.5416666666666666, total=   0.2s
[CV] random_state=55555, subsample=0.5, colsample_bytree=0.1, silent=True 
[CV] random_state=55555, subsample=0.5, colsample_bytree=0.1, silent=True 
[CV]  random_state=55555, subsample=0.4, col

...

[CV] random_state=55555, subsample=0.4, colsample_bytree=1.0, silent=True 
[CV]  random_state=55555, subsample=0.3, colsample_bytree=1.0, silent=True, score=0.5416666666666667, total=   0.5s
[CV]  random_state=55555, subsample=0.3, colsample_bytree=1.0, silent=True, score=0.6190476190476191, total=   0.5s
[CV] random_state=55555, subsample=0.4, colsample_bytree=1.0, silent=True 
[CV] random_state=55555, subsample=0.4, colsample_bytree=1.0, silent=True 
[CV]  random_state=55555, subsample=0.3, colsample_bytree=1.0, silent=True, score=0.4523809523809524, total=   0.5s
[CV] random_state=55555, subsample=0.5, colsample_bytree=1.0, silent=True 
[CV]  random_state=55555, subsample=0.3, colsample_bytree=1.0, silent=True, score=0.42857142857142855, total=   0.6s
[CV] random_state=55555, subsample=0.5, colsample_bytree=1.0, silent=True 
[CV]  random_state=55555, subsample=0.3, colsample_bytree=1.0, silent=True, score=0.4285714285714286, total=   0.6s
[CV] random_state=55555, subsample=0.5, cols

[Parallel(n_jobs=-1)]: Done 483 tasks      | elapsed:   34.7s


[CV]  random_state=55555, subsample=0.7, colsample_bytree=1.0, silent=True, score=0.6, total=   1.1s
[CV] random_state=55555, subsample=0.9, colsample_bytree=1.0, silent=True 
[CV]  random_state=55555, subsample=0.8, colsample_bytree=1.0, silent=True, score=0.4375, total=   1.2s
[CV] random_state=55555, subsample=0.9, colsample_bytree=1.0, silent=True 
[CV]  random_state=55555, subsample=0.8, colsample_bytree=1.0, silent=True, score=0.7142857142857143, total=   1.2s
[CV] random_state=55555, subsample=0.9, colsample_bytree=1.0, silent=True 
[CV]  random_state=55555, subsample=0.8, colsample_bytree=1.0, silent=True, score=0.6190476190476191, total=   1.1s
[CV] random_state=55555, subsample=1.0, colsample_bytree=1.0, silent=True 
[CV]  random_state=55555, subsample=0.8, colsample_bytree=1.0, silent=True, score=0.21428571428571427, total=   1.2s
[CV] random_state=55555, subsample=1.0, colsample_bytree=1.0, silent=True 
[CV]  random_state=55555, subsample=0.8, colsample_bytree=1.0, silent=T

[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   37.0s finished
/home/qiwei/anaconda2/envs/tensorflow-gpu-test/lib/python3.5/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.47393, std: 0.11457, params: {'random_state': 55555, 'subsample': 0.1, 'colsample_bytree': 0.1, 'silent': True},
  mean: 0.56869, std: 0.07862, params: {'random_state': 55555, 'subsample': 0.2, 'colsample_bytree': 0.1, 'silent': True},
  mean: 0.49845, std: 0.08451, params: {'random_state': 55555, 'subsample': 0.3, 'colsample_bytree': 0.1, 'silent': True},
  mean: 0.55000, std: 0.13477, params: {'random_state': 55555, 'subsample': 0.4, 'colsample_bytree': 0.1, 'silent': True},
  mean: 0.54738, std: 0.10351, params: {'random_state': 55555, 'subsample': 0.5, 'colsample_bytree': 0.1, 'silent': True},
  mean: 0.60155, std: 0.15498, params: {'random_state': 55555, 'subsample': 0.6, 'colsample_bytree': 0.1, 'silent': True},
  mean: 0.51869, std: 0.10328, params: {'random_state': 55555, 'subsample': 0.7, 'colsample_bytree': 0.1, 'silent': True},
  mean: 0.52964, std: 0.13840, params: {'random_state': 55555, 'subsample': 0.8, 'colsample_bytree': 0.1, 'silent': True},
  mean: 0.58893,

In [ ]:
###########################
#Grid search Tune subsample and colsample
#
#################################
param_test3 = {
             'reg_alpha':[i for i in range(0,3)],
             'reg_lambda':[i for i in range(1,100)],
    'silent':[True],
    'random_state':[r_seed]
}
cv = StratifiedKFold(n_splits = num_cv, random_state = r_seed, shuffle = True)

gsearch3 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=23, booster = 'gbtree',
                                                 max_depth = 3, min_child_weight=1, subsample = 0.4,
                                                 colsample_bytree = 0.9), 
  param_grid = param_test3, scoring="roc_auc",n_jobs=-1,iid=False, cv=cv,verbose=10)
gsearch3.fit(X,y)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

Fitting 5 folds for each of 297 candidates, totalling 1485 fits
[CV] random_state=55555, reg_lambda=1, silent=True, reg_alpha=0 ......
[CV] random_state=55555, reg_lambda=1, silent=True, reg_alpha=0 ......
[CV] random_state=55555, reg_lambda=1, silent=True, reg_alpha=0 ......
[CV] random_state=55555, reg_lambda=1, silent=True, reg_alpha=0 ......
[CV] random_state=55555, reg_lambda=1, silent=True, reg_alpha=0 ......
[CV] random_state=55555, reg_lambda=2, silent=True, reg_alpha=0 ......
[CV] random_state=55555, reg_lambda=2, silent=True, reg_alpha=0 ......
[CV] random_state=55555, reg_lambda=2, silent=True, reg_alpha=0 ......
[CV]  random_state=55555, reg_lambda=1, silent=True, reg_alpha=0, score=0.37500000000000006, total=   0.6s
[CV] random_state=55555, reg_lambda=2, silent=True, reg_alpha=0 ......
[CV]  random_state=55555, reg_lambda=1, silent=True, reg_alpha=0, score=0.9047619047619048, total=   0.6s
[CV] random_state=55555, reg_lambda=2, silent=True, reg_alpha=0 ......
[CV]  random_

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.7s


[CV] random_state=55555, reg_lambda=3, silent=True, reg_alpha=0 ......
[CV]  random_state=55555, reg_lambda=2, silent=True, reg_alpha=0, score=0.5238095238095238, total=   1.0s
[CV] random_state=55555, reg_lambda=4, silent=True, reg_alpha=0 ......
[CV]  random_state=55555, reg_lambda=3, silent=True, reg_alpha=0, score=0.375, total=   0.6s
[CV] random_state=55555, reg_lambda=4, silent=True, reg_alpha=0 ......
[CV]  random_state=55555, reg_lambda=2, silent=True, reg_alpha=0, score=0.8095238095238095, total=   0.7s
[CV] random_state=55555, reg_lambda=4, silent=True, reg_alpha=0 ......
[CV]  random_state=55555, reg_lambda=3, silent=True, reg_alpha=0, score=0.5, total=   0.7s
[CV] random_state=55555, reg_lambda=4, silent=True, reg_alpha=0 ......


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.3s


[CV]  random_state=55555, reg_lambda=3, silent=True, reg_alpha=0, score=0.8333333333333334, total=   0.7s
[CV]  random_state=55555, reg_lambda=2, silent=True, reg_alpha=0, score=0.42857142857142855, total=   0.8s
[CV] random_state=55555, reg_lambda=4, silent=True, reg_alpha=0 ......
[CV] random_state=55555, reg_lambda=5, silent=True, reg_alpha=0 ......
[CV]  random_state=55555, reg_lambda=3, silent=True, reg_alpha=0, score=0.9523809523809524, total=   0.8s
[CV]  random_state=55555, reg_lambda=3, silent=True, reg_alpha=0, score=0.39999999999999997, total=   0.7s
[CV] random_state=55555, reg_lambda=5, silent=True, reg_alpha=0 ......
[CV] random_state=55555, reg_lambda=5, silent=True, reg_alpha=0 ......
[CV]  random_state=55555, reg_lambda=4, silent=True, reg_alpha=0, score=0.39583333333333337, total=   0.6s
[CV] random_state=55555, reg_lambda=5, silent=True, reg_alpha=0 ......
[CV]  random_state=55555, reg_lambda=4, silent=True, reg_alpha=0, score=0.9523809523809524, total=   0.7s


...

[CV]  random_state=55555, reg_lambda=43, silent=True, reg_alpha=1, score=0.7857142857142857, total=   0.6s
[CV] random_state=55555, reg_lambda=45, silent=True, reg_alpha=1 .....
[CV]  random_state=55555, reg_lambda=44, silent=True, reg_alpha=1, score=0.22916666666666669, total=   0.6s
[CV] random_state=55555, reg_lambda=45, silent=True, reg_alpha=1 .....
[CV]  random_state=55555, reg_lambda=43, silent=True, reg_alpha=1, score=0.4857142857142857, total=   0.6s
[CV] random_state=55555, reg_lambda=45, silent=True, reg_alpha=1 .....
[CV]  random_state=55555, reg_lambda=44, silent=True, reg_alpha=1, score=0.5, total=   0.6s
[CV] random_state=55555, reg_lambda=45, silent=True, reg_alpha=1 .....
[CV]  random_state=55555, reg_lambda=44, silent=True, reg_alpha=1, score=0.4857142857142857, total=   0.6s
[CV] random_state=55555, reg_lambda=46, silent=True, reg_alpha=1 .....
[CV]  random_state=55555, reg_lambda=44, silent=True, reg_alpha=1, score=0.7857142857142857, total=   0.6s
[CV]  random_stat

[Parallel(n_jobs=-1)]: Done 745 tasks      | elapsed:  1.0min


[CV]  random_state=55555, reg_lambda=51, silent=True, reg_alpha=1, score=0.4857142857142857, total=   0.6s
[CV]  random_state=55555, reg_lambda=51, silent=True, reg_alpha=1, score=0.7857142857142857, total=   0.7s
[CV] random_state=55555, reg_lambda=53, silent=True, reg_alpha=1 .....
[CV] random_state=55555, reg_lambda=53, silent=True, reg_alpha=1 .....
[CV]  random_state=55555, reg_lambda=52, silent=True, reg_alpha=1, score=0.22916666666666669, total=   0.6s
[CV] random_state=55555, reg_lambda=53, silent=True, reg_alpha=1 .....
[CV]  random_state=55555, reg_lambda=52, silent=True, reg_alpha=1, score=0.7857142857142857, total=   0.6s
[CV] random_state=55555, reg_lambda=53, silent=True, reg_alpha=1 .....
[CV]  random_state=55555, reg_lambda=52, silent=True, reg_alpha=1, score=0.8571428571428572, total=   0.6s
[CV] random_state=55555, reg_lambda=54, silent=True, reg_alpha=1 .....
[CV]  random_state=55555, reg_lambda=52, silent=True, reg_alpha=1, score=0.4857142857142857, total=   0.6s
[C

[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  1.1min


[CV] random_state=55555, reg_lambda=60, silent=True, reg_alpha=1 .....
[CV]  random_state=55555, reg_lambda=59, silent=True, reg_alpha=1, score=0.5, total=   0.6s
[CV] random_state=55555, reg_lambda=61, silent=True, reg_alpha=1 .....
[CV]  random_state=55555, reg_lambda=59, silent=True, reg_alpha=1, score=0.7857142857142857, total=   0.6s
[CV] random_state=55555, reg_lambda=61, silent=True, reg_alpha=1 .....
[CV]  random_state=55555, reg_lambda=59, silent=True, reg_alpha=1, score=0.4857142857142857, total=   0.6s
[CV] random_state=55555, reg_lambda=61, silent=True, reg_alpha=1 .....
[CV]  random_state=55555, reg_lambda=60, silent=True, reg_alpha=1, score=0.22916666666666669, total=   0.6s
[CV] random_state=55555, reg_lambda=61, silent=True, reg_alpha=1 .....
[CV]  random_state=55555, reg_lambda=60, silent=True, reg_alpha=1, score=0.8571428571428572, total=   0.7s
[CV] random_state=55555, reg_lambda=61, silent=True, reg_alpha=1 .....
[CV]  random_state=55555, reg_lambda=60, silent=True,

In [ ]:
#################################################################
# Use random search to train a XGBoost model
# with bagging
###################################################
#est = XGBClassifier()
#param_test_rand = {
#            'learning_rate':[0.4,0.5,0.7],
#            'n_estimators':[i for i in range(12,20)],
#            #'n_estimators':[i for i in range(1,10)],
#            'max_depth':[i for i in range(12,20)],
#            'min_child_weight':[i for i in range(0,5)],
#             'booster':['gbtree','gblinear','dart'],
#             'subsample':[i/100.0 for i in range(70,90)],
#             'colsample_bytree':[i/100.0 for i in range(40,60)],
#             'reg_alpha':[i for i in range(0,5)],
#             'reg_lambda':[i for i in range(100,120)],
#             'silent':[True],
#             'random_state':[42]   

 #'learning_rate':[0.1, 0.01, 0.001, 0.0001],
 #'n_estimators':[i for i in range(1,1000,10)],
 #'n_estimators':[i for i in range(1,10)],
 #'max_depth':[i for i in range(1,30)],
 #'min_child_weight':[i for i in range(0,10)],
 #'booster':['gbtree','gblinear','dart'],
 #'subsample':[i/100.0 for i in range(1,100,1)],
 #'colsample_bytree':[i/100.0 for i in range(1,100,1)],
 #'reg_alpha':[i for i in range(0,10)],
 #'reg_lambda':[i for i in range(0,100)],
 #'silent':[True],
 #'random_state':[42]
#}

#gb_estimator, gb_parameters, gb_cv_results = hypertuning_rscv(est, param_test_rand, 6000, X, y)

#print(gb_parameters)

#print('Hyper-tuned model score :')
#print(gb_ht_score*100)

#print('Hyper-tuned model training mean :')
#print(gb_train_mean)
#print('Hyper-tuned model training std :')
#print(gb_train_std)

#print('Hyper-tuned model training mean for split1 :')
#print(gb_train_mean_sp0)

#print('Hyper-tuned model training mean for split2 :')
#print(gb_train_mean_sp1)

#print('Hyper-tuned model training mean for split3 :')
#print(gb_train_mean_sp2)


In [ ]:
#print(gb_cv_results["split1_train_score"])
#print(gb_cv_results["split1_test_score"])

In [ ]:
# training a XGBoost model
#xgb = XGBClassifier(
#                    learning_rate =0.01,
#                    n_estimators=495,
#                    booster = 'gblinear',
#                    max_depth = 22,
#                     min_child_weight=0,
#                     subsample=0.85,
#                     colsample_bytree=0.13,
#                     objective= 'binary:logistic',
#                    reg_lambda = 12,
#                    reg_alpha =0,
#                    random_state=42)
# if using the randomSearch method
#xgb = gb_estimator

# if using GridSearch method
xgb = gsearch3.best_estimator_

cv = StratifiedKFold(n_splits = num_cv, random_state = r_seed, shuffle = True)

#xgb_scores = cross_val_score(xgb, X, y, cv = cv, scoring=multiclass_score)
#xgb_pred = cross_val_predict(xgb, X, y, cv = 3)

##!!!!
#notice that mean of auroc of each fold is different from the auroc calculated by all the predicted probability
#svm_scores = cross_val_score(svm_model_linear, X, y, cv = cv, scoring=multiclass_score)
y_xgb_prob = cross_val_predict(xgb, X, y, cv = cv, method = 'predict_proba')

# calculate the auroc by directly using the multiclass_roc_auc_score scorer
#xgb_multiclass_auroc = multiclass_roc_auc_score(y, y_xgb_prob, average="weighted")

# calculate the auroc by directly using the binaryiclass_roc_auc_score scorer
xgb_multiclass_auroc = binary_class_roc_auc_score(y, y_xgb_prob[:,1], average="weighted")

print(xgb)
print("Auroc across all folds: %0.5f" % (xgb_multiclass_auroc))

#for train_index0, test_index0 in cv.split(X, y):
#     print("TRAIN:", train_index0, "TEST:", test_index0)
        
#print(compress_path)


In [ ]:
#print("Predicted labels are:")
#print(xgb_pred)

#output predicted labels
#XGboost
#xgb_pred_df = pd.DataFrame(xgb_pred)
#xgb_pred_file = os.path.join("predicted_labels", "xgb_pred.tsv")
#xgb_pred_df.to_csv(xgb_pred_file, sep='\t')

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

cv = StratifiedKFold(n_splits = num_cv, random_state = r_seed, shuffle = True)
y_xgb_pred = cross_val_predict(xgb, X, y, cv = cv)
xgb_conf_mat = confusion_matrix(y,y_xgb_pred)

#from sklearn.metrics import roc_auc_score
#print(roc_auc_score(y, y_xgb_pred))

print(xgb_conf_mat)

In [ ]:
#count the importance of features, and see actually how many are useful
np.count_nonzero(xgb.feature_importances_)

In [ ]:
#######################################
#plot feature importance
###########################################
import xgboost
xgboost.plot_importance(xgb)
plt.rcParams['figure.figsize'] = [10, 30]
#plt.savefig('counts_data/(0806)Feature_Importance(deep10+3L_0.1t_0.2var)(BLCA,seed9).png')
plt.show()

In [ ]:
################################
#################################
###################################

In [ ]:
########################################
#print out binary class roc auc figure
############################################
fpr, tpr, threshold = metrics.roc_curve(y,y_xgb_prob[:,1])
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
########################################
#print out multiclass roc auc figure
############################################
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

cv = StratifiedKFold(n_splits = 3, random_state = 42)
y_xgb_prob = cross_val_predict(xgb, X, y, cv = cv, method = 'predict_proba')

import matplotlib.pyplot as plt
plt.figure(figsize = (10, 8))
plot_multiclass_roc_auc(y, y_xgb_prob, n_classes = 2, title = "xgb, SARC0.2")
#plt.savefig('result/3_classes/(0606)Multiclass_roc_auc_xgb(SARC0.2).png')
plt.show()